# 영화 평점 분석 실습

In [2]:
import pandas as pd
import numpy as np

## 1. 영화 평점 데이터 적재 및 전처리

In [3]:
# 사용자 데이터 읽어오기
users = pd.read_csv('data/movielens/users.dat', sep = '::', engine = 'python',
                   names = ['사용자아이디', '성별','연령','직업','지역'])
users.head()

,사용자아이디,성별,연령,직업,지역
0,1,F,1,10,48067
1,2,M,56,16,70072
2,3,M,25,15,55117
3,4,M,45,7,02460
4,5,M,25,20,55455


In [4]:
# 평점 데이터 읽어오기
ratings = pd.read_csv('data/movielens/ratings.dat', sep = '::', engine = 'python',
                   names = ['사용자아이디', '영화아이디','평점','타임스탬프'])
ratings.head()

,사용자아이디,영화아이디,평점,타임스탬프
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


In [5]:
# 영화데이터 읽어오기
movies = pd.read_csv('data/movielens/movies.dat', sep = '::', engine = 'python',
                   names = ['영화아이디','영화제목','장르'], encoding = 'latin-1')
movies.head()

,영화아이디,영화제목,장르
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy


In [12]:
#3개의 데이터프레임을 하나로 합치기
df = pd.merge(ratings,movies,how='outer',on='영화아이디').merge(users,how='outer',on='사용자아이디')
df.head()

,사용자아이디,영화아이디,평점,타임스탬프,영화제목,장르,성별,연령,직업,지역
0,1.0,1193,5.0,978300760.0,One Flew Over the Cuckoo's Nest (1975),Drama,F,1.0,10.0,48067
1,1.0,661,3.0,978302109.0,James and the Giant Peach (1996),Animation|Children's|Musical,F,1.0,10.0,48067
2,1.0,914,3.0,978301968.0,My Fair Lady (1964),Musical|Romance,F,1.0,10.0,48067
3,1.0,3408,4.0,978300275.0,Erin Brockovich (2000),Drama,F,1.0,10.0,48067
4,1.0,2355,5.0,978824291.0,"Bug's Life, A (1998)",Animation|Children's|Comedy,F,1.0,10.0,48067


## 2. 보고 싶은 영화 찾기
영화들의 평점 평균을 구하여, 사람들에게 인정받는 (평점이 높은) 영화 찾기

In [ ]:
# 영화들의 평점 평균을 구하여, 평점이 높은 영화 찾기 

In [28]:
df.groupby('영화제목').평점.mean().fillna(0).sort_values(0,ascending=False).head(20)

영화제목
Schlafes Bruder (Brother of Sleep) (1995)                              5.000000
Lured (1947)                                                           5.000000
Song of Freedom (1936)                                                 5.000000
Gate of Heavenly Peace, The (1995)                                     5.000000
Ulysses (Ulisse) (1954)                                                5.000000
Bittersweet Motel (2000)                                               5.000000
Follow the Bitch (1998)                                                5.000000
Smashing Time (1967)                                                   5.000000
Baby, The (1973)                                                       5.000000
One Little Indian (1973)                                               5.000000
I Am Cuba (Soy Cuba/Ya Kuba) (1964)                                    4.800000
Lamerica (1994)                                                        4.750000
Apple, The (Sib) (1998)            

평균 평점이 만점인 영화들이 최상위에 위치함. 
일반적으로 평점이 만점인 경우는 대부분 평점의 개수가 매우 적은 경우이므로, 이를 확인하기 위해 평점의 개수도 함께 구해본다. 

In [40]:
df.pivot_table(index='영화제목',values='평점',aggfunc=['count',np.mean]).sort_values(('mean','평점'),ascending=False).head(20)

,count,mean
,평점,평점
Ulysses (Ulisse) (1954),1,5.000000
Lured (1947),1,5.000000
Follow the Bitch (1998),1,5.000000
Bittersweet Motel (2000),1,5.000000
Song of Freedom (1936),1,5.000000
One Little Indian (1973),1,5.000000
Smashing Time (1967),2,5.000000
Schlafes Bruder (Brother of Sleep) (1995),1,5.000000
"Gate of Heavenly Peace, The (1995)",3,5.000000


## [실습 #1] 여자들이 좋아하는 영화 찾기 
여성 평점이 4.0 이상이고 여성 평점의 개수가 500개 이상인 영화

In [60]:
df1 = df[df.성별=='F'].pivot_table(index='영화제목',values='평점',aggfunc=['count',np.mean])
sz1 = df1[(df1[('count','평점')]>=500) & (df1[('mean','평점')] >= 4)]
sz1

,count,mean
,평점,평점
영화제목,,
American Beauty (1999),946,4.238901
Being John Malkovich (1999),569,4.159930
Braveheart (1995),546,4.016484
Casablanca (1942),505,4.300990
E.T. the Extra-Terrestrial (1982),601,4.089850
Fargo (1996),657,4.217656
Forrest Gump (1994),644,4.045031
L.A. Confidential (1997),566,4.106007


## [실습 #2] 실습 #1에서 구한 영화(여성인기영화)의 장르를 분석해 보자.
여성인기영화의 장르 통계 구하기

예를 들어, 여성인기영화 중 Drama 장르의 영화는 10개, Action 영화는 3개, ...

In [75]:
dic2 = {}
for i in movies[movies.영화제목.isin(sz1.index)].장르 :
    for j in i.split('|'):
        dic2[j]=dic2.get(j,0)+1
dic2

{'Animation': 1,
 "Children's": 3,
 'Comedy': 6,
 'Action': 7,
 'Drama': 12,
 'War': 6,
 'Adventure': 5,
 'Fantasy': 2,
 'Sci-Fi': 4,
 'Crime': 3,
 'Romance': 4,
 'Thriller': 5,
 'Musical': 1,
 'Film-Noir': 1,
 'Mystery': 1}

## [실습 #3] 남자와 여자의 호불호가 크게 갈리는 영화 10개 찾기
전체 평점의 개수가 500개 이상인 영화만 대상으로 함.

In [86]:
sz3 = df.groupby('영화제목').평점.count()[df.groupby('영화제목').평점.count()>500]
sz3

영화제목
10 Things I Hate About You (1999)         700
101 Dalmatians (1961)                     565
12 Angry Men (1957)                       616
13th Warrior, The (1999)                  750
20,000 Leagues Under the Sea (1954)       575
                                         ... 
X-Files: Fight the Future, The (1998)     996
X-Men (2000)                             1511
You've Got Mail (1998)                    838
Young Frankenstein (1974)                1193
Young Guns (1988)                         562
Name: 평점, Length: 617, dtype: int64

In [93]:
df3 = df[df.영화제목.isin(sz3.index)].groupby(['영화제목','성별']).평점.mean().unstack('성별')
df3

성별,F,M
영화제목,,
10 Things I Hate About You (1999),3.646552,3.311966
101 Dalmatians (1961),3.791444,3.500000
12 Angry Men (1957),4.184397,4.328421
"13th Warrior, The (1999)",3.112000,3.168000
"20,000 Leagues Under the Sea (1954)",3.670103,3.709205
...,...,...
"X-Files: Fight the Future, The (1998)",3.489474,3.493797
X-Men (2000),3.682310,3.851702
You've Got Mail (1998),3.542424,3.275591


In [98]:
np.abs(df3.F-df3.M).sort_values()[::-1].head(10)

영화제목
Dirty Dancing (1987)                      0.830782
Good, The Bad and The Ugly, The (1966)    0.726351
Dumb & Dumber (1994)                      0.638608
Evil Dead II (Dead By Dawn) (1987)        0.611985
Grease (1978)                             0.608224
Caddyshack (1980)                         0.573602
Animal House (1978)                       0.538286
Exorcist, The (1973)                      0.529605
Rocky Horror Picture Show, The (1975)     0.512885
Big Trouble in Little China (1986)        0.497078
dtype: float64

## [실습 #4] 연령대 별로 영화 평점 분석하기
연령대(10대 미만, 10대, 20대, ...50대) 컬럼을 추가한 후, 영화별 연령대별 영화평점 구하기

In [170]:
df['연령대'] = ((np.trunc(df.연령/10)*10).astype('str')+'대').astype('category')\
.cat.rename_categories(['10대 미만', '10대', '20대', '30대', '40대', '50대', 'nan'])

In [171]:
df.groupby('연령대').평점.mean().dropna()

연령대
10대 미만    3.549520
10대       3.507573
20대       3.545235
30대       3.618162
40대       3.638062
50대       3.732677
Name: 평점, dtype: float64